# Cleaning charter school data using pandas

![Panda school](http://78.media.tumblr.com/885b93160e4a1d0ab595ade6434110d4/tumblr_mr8dc2FQG21raju2zo1_1280.jpg)

The [NCES CCD Public School Universe Surveys](https://nces.ed.gov/ccd/pubschuniv.asp) include data on enrollments, free/reduced price lunch, staff size, etc. for all public schools in the US ranging from the years 1986 to 2006. I have merged information on school status, address, reconstitution, and charter school status for a range of years  the years of 1998-99 (when charter school flags were first included in these data) to 2016-17 (the most recent data, which is preliminary as of January 2018) with the full data for the year 2014-15--the most recent year of full school data as of Fall 2017 (although as of Jan. 2018 full data is now available for the year 2015).

This notebook uses the Pandas library in Python to read in the full data set (see `https://github.com/jhaber-zz/Charter-school-identities/data/pubschools_merged_2014.csv`), narrow it down to only those schools documented to be charter schools in 2014-15, and drop any schools that are closed or inactive in 2014. 

As of early January 2018, steps to improve this process are:
- Consolidating charter status flags, which for some public schools are mysteriously inconsistent across years (even accounting for reconstitution flags)
- Gathering URLs and collecting data for schools opened in 2015 or 2016
- Merging and using the full 2015-16 data set

In [1]:
import pandas, csv, os

In [2]:
dir_prefix = '/home/jovyan/work/Charter-school-identities/'

In [3]:
data = pandas.read_json(dir_prefix + 'data/school_parser_temp.json')
data

,ADDRESS14,ADDRESS16,AE,AEOFFERED,ALL_MTH00NUMVALID_1415,ALL_MTH00PCTPROF_1415,ALL_RLA00NUMVALID_1415,ALL_RLA00PCTPROF_1415,AM,AMALF,...,YEAR_CLOSED,YEAR_OPENED,duplicate_flag,folder_name,html_file_count,ideology_text,keywords_text,parse_error_flag,webtext,wget_fail_flag
0,"1010 Fourth Avenue, Bethel AK 99559","1010 Fourth Avenue, Bethel AK 99559",-2.0,N,73.0,20-24,74.0,15-19,175.0,102.0,...,,1999.0,0.0,Ayaprun_Elitnaurvik_AK,1.0,[],[],0.0,"[¥H3§6""×SiU8@FU ̈Ð a+c-¦Þß¶24Ìç.ÝÒ'®...",0.0
1,"410 Schoenbar, Ketchikan AK 99901","410 Schoenbar, Ketchikan AK 99901",-2.0,N,126.0,40-44,126.0,40-44,65.0,33.0,...,,1998.0,0.0,Ketchikan_Charter_School_AK,61.0,"[Sign In\n, KCS students will be working with ...","[KCS School Creed\n, KCS School Creed\n, KCS S...",0.0,"[Skip to Main Content\n, District Home\n, Sele...",0.0
2,"410 Schoenbar Road, Ketchikan AK 99901","410 Schoenbar Road, Ketchikan AK 99901",-2.0,N,88.0,35-39,88.0,50-54,56.0,26.0,...,,2003.0,0.0,Tongass_School_of_Arts_and_Sciences_Charter_Sc...,1.0,"[Music\n, cook\n, . We offer a full-day classr...","[Our Vision\n, mission statement\n, Our School...",0.0,"[Tongass School of Arts and Sciences\n, Home\n...",0.0
3,"1705 W 32nd Ave, Anchorage AK 99517","1705 W 32nd Ave, Anchorage AK 99517",-2.0,N,197.0,65-69,197.0,70-74,11.0,7.0,...,,1998.0,0.0,Aquarian_Charter_School_AK,49.0,"[School Data\n, Activity Interest Survey\n, A...",[Aquarian students explore topics outside of s...,0.0,"[Main\n, Main\n, Anchorage school district\n, ...",0.0
4,"401 E Fireweed Lane Suite 100, Anchorage AK 99503","401 E Fireweed Lane Suite 100, Anchorage AK 99503",-2.0,N,210.0,31,209.0,54,28.0,15.0,...,,1998.0,0.0,Family_Partnership_Charter_School_AK,1.0,[],[],0.0,"[1⁄2ÐDp1⁄430 ́Ç_,_·sõ°`Ö§ï§hÚï3ÎâDxö...",0.0
5,"4802 Bryn MAWR Court, Anchorage AK 99508","4802 Bryn MAWR Court, Anchorage AK 99508",-2.0,N,153.0,15-19,154.0,40-44,13.0,10.0,...,,2005.0,0.0,Winterberry_School_AK,1.0,"[What Is Waldorf Education?\n, Anchorage Waldo...","[Mission\n, Curriculum\n]",0.0,"[Menu\n, Home\n, About\n, Mission\n, Leadershi...",0.0
6,"10901 Mausel St Suite 101, Eagle River AK 99577","10901 Mausel St Suite 101, Eagle River AK 99577",-2.0,N,92.0,70-74,92.0,75-79,2.0,1.0,...,,2005.0,0.0,Eagle_Academy_Charter_School_AK,5.0,[notice. The Anchorage School District is an e...,[Eagle Academy provides students with an excel...,0.0,"[Main\n, Anchorage School District\n, Educatin...",0.0
7,"400 W Northern Lights Blvd, #9, Anchorage AK 9...","400 W Northern Lights Blvd #9, Anchorage AK 99503",-2.0,N,148.0,20-24,148.0,40-44,22.0,8.0,...,,2003.0,0.0,Frontier_Charter_School_AK,25.0,[Í4>kFÃ3¿Ýö5N`ÑmPFêμÜ¶§îþyõJï1⁄4Þ!HUÇ@ÕR-¿Ü...,[Í4>kFÃ3¿Ýö5N`ÑmPFêμÜ¶§îþyõJï1⁄4Þ!HUÇ@ÕR-¿Ü...,0.0,[Í4>kFÃ3¿Ýö5N`ÑmPFêμÜ¶§îþyõJï1⁄4Þ!HUÇ@ÕR-¿Ü...,0.0
8,"5530 E Northern Lights Suite 1, Anchorage AK 9...","5530 E Northern Lights Suite 1, Anchorage AK 9...",-2.0,N,126.0,20-24,127.0,50-54,10.0,7.0,...,,2003.0,0.0,Highland_Tech_High_Charter_School_AK,33.0,"[School Data\n, Activity Interest Survey\n, P...","[Curriculum\n, Curriculum\n, Curriculum\n, Cur...",0.0,"[Main\n, Main\n, Anchorage school district\n, ...",0.0
9,"650 W International Airport Rd, Anchorage AK 9...","650 W International Airport Rd, Anchorage AK 9...",-2.0,N,197.0,35-39,197.0,45-49,8.0,5.0,...,,2007.0,0.0,Rilke_Schule_Charter_School_AK,1.0,[],[],0.0,"[Skip to main content\n, State of Alaska\n, my...",0.0


In [7]:
data = pandas.read_csv(dir_prefix + 'data/pubschools_merged_2014.csv', encoding='Latin1') # usecols=['SCHOOL_YEAR','STATENAME','ST','SCH_NAME','NCESSCH','LSTREET1', 'LSTREET2', 'LSTREET3', 'LCITY', 'LSTATE','LZIP','SY_STATUS','SY_STATUS_TEXT','CHARTER_TEXT','GSLO','GSHI','LEVEL']
print("Original length of data is " + str(len(data)))

data = data[data.CHARTER_TEXT=='Yes']
print("Length of data after clearing non-charters is " + str(len(data)))

data = data.drop(data[(data.SY_STATUS_TEXT=='Closed')|(data.SY_STATUS_TEXT=='Inactive')|(data.SY_STATUS_TEXT=='Future')].index) 
print("Length of data after clearing closed charters is " + str(len(data)))

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,3,4,5,7,8,10,12,13,14,15,16,17,21,22,23,24,25,29,30,32,33,34,35,36,38,40,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,68,270,271,272,273,353,354,355,356,357,358,359,360,362,363,364,366,367,369,370,372,373,375,376,378,379,381,382,384,385,387,388,390,391,393,394,396,397,399,400,402,403,404,405,407,409,410,412,414,415,417,419,420,422,423,424,426,434,437,438,439,442,444,445,446,447,448,449,450,452) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Original length of data is 136819
Length of data after clearing non-charters is 7304
Length of data after clearing closed charters is 6972


In [ ]:
# List of columns to drop from the data to keep things parsimonious:
drop_varlist = ["Unnamed: 0", "ISMEMPUP", "ISPELM", "ISPFEMALE", "ISPWHITE", \
                "PKOFFERED", "KGOFFERED", "G1OFFERED", "G2OFFERED", "G3OFFERED", "G4OFFERED", "G5OFFERED", "G6OFFERED", "G7OFFERED", "G8OFFERED", "G9OFFERED", "G10OFFERED", "G11OFFERED", "G12OFFERED", \
                "PK", "KG", "G01", "G02", "G03", "G04", "G05", "G06", "G07", "G08", "G09", "G10", "G11", "G12", "G13", "UG", \
                "AMPKM", "AMPKF", "ASPKM", "ASPKF", "HIPKM", "HIPKF", "BLPKM", "BLPKF", "WHPKM", "WHPKF", "HPPKM", "HPPKF", "TRPKM", "TRPKF", \
                "AMKGM", "AMKGF", "ASKGM", "ASKGF", "HIKGM", "HIKGF", "BLKGM", "BLKGF", "WHKGM", "WHKGF", "HPKGM", "HPKGF", "TRKGM", "TRKGF", \
                "AM01M", "AM01F", "AS01M", "AS01F", "HI01M", "HI01F", "BL01M", "BL01F", "WH01M", "WH01F", "HP01M", "HP01F", "TR01M", "TR01F", \
                "AM02M", "AM02F", "AS02M", "AS02F", "HI02M", "HI02F", "BL02M", "BL02F", "WH02M", "WH02F", "HP02M", "HP02F", "TR02M", "TR02F", \
                "AM03M", "AM03F", "AS03M", "AS03F", "HI03M", "HI03F", "BL03M", "BL03F", "WH03M", "WH03F", "HP03M", "HP03F", "TR03M", "TR03F", \
                "AM04M", "AM04F", "AS04M", "AS04F", "HI04M", "HI04F", "BL04M", "BL04F", "WH04M", "WH04F", "HP04M", "HP04F", "TR04M", "TR04F", \
                "AM05M", "AM05F", "AS05M", "AS05F", "HI05M", "HI05F", "BL05M", "BL05F", "WH05M", "WH05F", "HP05M", "HP05F", "TR05M", "TR05F", \
                "AM06M", "AM06F", "AS06M", "AS06F", "HI06M", "HI06F", "BL06M", "BL06F", "WH06M", "WH06F", "HP06M", "HP06F", "TR06M", "TR06F", \
                "AM07M", "AM07F", "AS07M", "AS07F", "HI07M", "HI07F", "BL07M", "BL07F", "WH07M", "WH07F", "HP07M", "HP07F", "TR07M", "TR07F", \
                "AM08M", "AM08F", "AS08M", "AS08F", "HI08M", "HI08F", "BL08M", "BL08F", "WH08M", "WH08F", "HP08M", "HP08F", "TR08M", "TR08F", \
                "AM09M", "AM09F", "AS09M", "AS09F", "HI09M", "HI09F", "BL09M", "BL09F", "WH09M", "WH09F", "HP09M", "HP09F", "TR09M", "TR09F", \
                "AM10M", "AM10F", "AS10M", "AS10F", "HI10M", "HI10F", "BL10M", "BL10F", "WH10M", "WH10F", "HP10M", "HP10F", "TR10M", "TR10F", \
                "AM11M", "AM11F", "AS11M", "AS11F", "HI11M", "HI11F", "BL11M", "BL11F", "WH11M", "WH11F", "HP11M", "HP11F", "TR11M", "TR11F", \
                "AM12M", "AM12F", "AS12M", "AS12F", "HI12M", "HI12F", "BL12M", "BL12F", "WH12M", "WH12F", "HP12M", "HP12F", "TR12M", "TR12F", \
                "AM13M", "AM13F", "AS13M", "AS13F", "HI13M", "HI13F", "BL13M", "BL13F", "WH13M", "WH13F", "HP13M", "HP13F", "TR13M", "TR13F", \
                "AMUGM", "AMUGF", "ASUGM", "ASUGF", "HIUGM", "HIUGF", "BLUGM", "BLUGF", "WHUGM", "WHUGF", "HPUGM", "HPUGF", "TRUGM", "TRUGF", \
                "AMAEM", "AMAEF", "ASAEM", "ASAEF", "HIAEM", "HIAEF", "BLAEM", "BLAEF", "WHAEM", "WHAEF", "HPAEM", "HPAEF", "TRAEM", "TRAEF"]

In [ ]:
print("Number of columns in big data set is " + str(len(data.keys())))
print("Number of columns to drop is " + str(len(drop_varlist)))
print("Dropping these should get us down to " + str(int(str(len(data.keys())))-int(str(len(drop_varlist)))) + " (the difference)")

In [ ]:
data = data.drop(drop_varlist,axis=1)  
# See options here: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

print("Number of columns in big data set has been reduced to " + str(len(data.keys())))

In [ ]:
data

In [ ]:
#data['ADDRESS14'].replace(regex=True,inplace=True,to_replace=r'.0',value=r'')
data['ADDRESS14'].replace(regex=True,inplace=True,to_replace=r' , ',value=r', ')
data['ADDRESS14'] = data['ADDRESS14'].str.strip(' ')
data['ADDRESS14'] = data['ADDRESS14'].str.strip('.0')
data['ADDRESS14'].iloc[0]

In [ ]:
#data['ADDRESS16'].replace(regex=True,inplace=True,to_replace=r'.0',value=r'')
data['ADDRESS16'].replace(regex=True,inplace=True,to_replace=r' , ',value=r', ')
data['ADDRESS16'] = data['ADDRESS16'].str.strip(' ')
data['ADDRESS16'] = data['ADDRESS16'].str.strip('.0')
data['ADDRESS16'].iloc[0]

In [ ]:
data[['ADDRESS14','ADDRESS16']].iloc[1480]

In [ ]:
data[['ADDRESS14','ADDRESS16']]

In [ ]:
# data.to_csv(dir_prefix + 'data/charter_merged_2014.csv',encoding='Latin1',index=False)

In [4]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [ ]:
df = pd.DataFrame.from_dict(dicts_list)
df.head()

In [9]:
data = pandas.read_csv(dir_prefix + 'data/charter_merged_2014.csv', encoding='Latin1')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6972 entries, 0 to 6971
Columns: 180 entries, SURVYEAR to ADDRESS16
dtypes: float64(86), object(94)
memory usage: 9.6+ MB


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14,109,112,115,132,175) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
data.describe()

,FIPST,LEAID,SCHID,NCESSCH,MZIP,MZIP4,PHONE,LZIP14,LZIP4,UNION,...,PVI2014,LATCODE,LONGCODE,CONUM,CD,LOCALE,METMIC,ALL_MTH00NUMVALID_1415,ALL_RLA00NUMVALID_1415,LZIP16
count,6972.000000,6.972000e+03,6972.000000,6.972000e+03,6972.000000,2734.000000,6.972000e+03,6972.000000,3079.000000,6972.000000,...,6727.000000,6858.000000,6858.000000,6858.000000,6858.000000,6858.000000,6858.000000,6284.000000,6284.000000,6545.000000
mean,23.983936,2.404996e+06,5491.411647,2.404996e+11,60868.481928,4038.390636,5.778663e+09,60887.338927,4123.528418,0.098967,...,-6.278728,36.534483,-96.613998,24156.267425,2420.659813,18.690143,1.029892,217.874761,222.829249,60727.400917
std,16.934363,1.688813e+06,4241.496770,1.688813e+11,28308.792826,2556.950010,2.393566e+09,28319.657689,2543.128806,2.130168,...,16.479113,5.654503,17.057601,16949.428694,1689.213965,10.045010,0.285489,289.746652,292.885915,28641.236592
min,2.000000,2.000010e+05,11.000000,2.000010e+10,1022.000000,1.000000,2.012003e+09,1022.000000,1.000000,0.000000,...,-43.000000,19.039730,-165.405470,2020.000000,200.000000,11.000000,0.000000,1.000000,1.000000,1022.000000
25%,6.000000,6.379275e+05,1911.750000,6.379276e+10,33763.500000,2065.500000,3.309411e+09,33765.000000,2142.000000,0.000000,...,-19.000000,32.803884,-112.053363,6087.000000,643.000000,11.000000,1.000000,64.000000,68.000000,33612.000000
50%,22.000000,2.201170e+06,4070.500000,2.201170e+11,70118.000000,3378.500000,6.022392e+09,70117.500000,3438.000000,0.000000,...,-6.000000,36.737500,-93.234533,24033.000000,2403.000000,13.000000,1.000000,148.000000,152.000000,67357.000000
75%,39.000000,3.901375e+06,8344.250000,3.901375e+11,85642.250000,5435.000000,8.012996e+09,85647.000000,5529.500000,0.000000,...,7.000000,40.785521,-81.952231,39095.000000,3911.000000,21.000000,1.000000,291.000000,294.000000,85711.000000
max,56.000000,5.601980e+06,30008.000000,5.601980e+11,99901.000000,9998.000000,9.899211e+09,99901.000000,9998.000000,60.000000,...,32.000000,64.847827,-69.629684,56021.000000,5600.000000,43.000000,2.000000,7147.000000,6910.000000,99901.000000


In [ ]:
data["folder_name"] = re.sub(" ","_",(school[NAME_var]+" "+school[ADDR_var][-8:-6]))
data["wget_fail_flag"] = 

In [ ]:
df.info(memory_usage="deep") # get accurate sense of memory usage
df.describe()

In [16]:
from tqdm import tqdm
tqdm.pandas(desc="my bar!")

In [17]:
import tqdm
for i in tqdm.trange(int(1e8), miniters=int(1e6), ascii=True,
                     desc="cool", dynamic_ncols=True):
    pass

cool:  22%|##2       | 22000000/100000000 [00:05<00:17, 4359146.51it/s]


KeyboardInterrupt: 

In [ ]:
fipst = df.ALL_RLA00PCTPROF_1415
print(fipst.head(7))

fipst_cat = fipst.astype('category')
print(fipst_cat.head(7))

print(fipst_cat.head(7).cat.codes)

In [ ]:
print(mem_usage(fipst))
print(mem_usage(fipst_cat))

In [ ]:
converted_df = pd.DataFrame()

for col in df.columns:
    num_unique_values = len(df[col].unique())
    num_total_values = len(df[col])
    if num_unique_values / num_total_values < 0.5:
        converted_df.loc[:,col] = df[col].astype('category')
    else:
        converted_df.loc[:,col] = df[col]

In [ ]:
print(mem_usage(df))
print(mem_usage(converted_df))

compare_df = pd.concat([df.dtypes,converted_df.dtypes],axis=1)
compare_df.columns = ['before','after']
compare_df.apply(pd.Series.value_counts)